<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Все-функции" data-toc-modified-id="Все-функции-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Все функции</a></span><ul class="toc-item"><li><span><a href="#Функции-перевода-изображения-в-монохромный-формат-и-resize." data-toc-modified-id="Функции-перевода-изображения-в-монохромный-формат-и-resize.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Функции перевода изображения в монохромный формат и resize.</a></span></li><li><span><a href="#Функции-разрезания-на-ленты" data-toc-modified-id="Функции-разрезания-на-ленты-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Функции разрезания на ленты</a></span><ul class="toc-item"><li><span><a href="#разрезание-на-ленты-на-примере-одной-матрицы" data-toc-modified-id="разрезание-на-ленты-на-примере-одной-матрицы-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>разрезание на ленты на примере одной матрицы</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time as tm

from skimage import io, color, img_as_float
from scipy.ndimage import map_coordinates
import cv2 


# Все функции

## Функции перевода изображения в монохромный формат и resize. 
А также функции умножения матрицы на транспонированную и различных нормализаций. 
Вдруг понадобятся ;-) 

In [2]:
t1_itogo = tm.time()


def monochrome(rgb_image):
    coeffs = np.array([0.299, 0.587, 0.114]) #!!! Не переводим элементы вектора в формат unit8
    return rgb_image @ coeffs


def resize_and_monochrome (picture, k):
    new_size = (round(picture.shape[1]/k), round(picture.shape[0]/k))
    picture = cv2.resize(picture, new_size, interpolation=cv2.INTER_AREA)
    picture_monochrome = monochrome(picture)
    return picture, picture_monochrome



In [3]:
def normalize_image(im):   
    im_res = im - np.min(im)
    return (1 / np.max(im_res)) * im_res

def norm_columns(x):
    columns_sums = np.sum(x, axis=0)
    columns_sums[columns_sums == 0] = np.finfo(float).eps
    # Убедимся, что нормализация применяется корректно
    return x / columns_sums[np.newaxis, :]

# поправила norm_rows, а то там появляется деление на ноль
def norm_rows(x):
    row_sums = np.sum(x, axis=1)
    # Замена нулей на очень маленькое число, чтобы избежать деления на ноль
    row_sums[row_sums == 0] = np.finfo(float).eps
    return x / row_sums[:, np.newaxis]


def trans_mult(A):
    result = normalize_image(np.dot(monochrome(A), np.transpose(monochrome(A))))
    return result

def trans_mult_norm(A):
    result = normalize_image(np.dot(norm_rows(monochrome(A)), norm_columns(np.transpose(monochrome(A)))))
    return result

def trans_mult_norm_plus_1(A):
    B = A+1
    result = normalize_image(np.dot(norm_rows(monochrome(B)), norm_columns(np.transpose(monochrome(B)))))
    return result

def normalize_image_signed(img):
    img1 = img - np.min(img)
    img2 = (2 / np.max(img1)) * img1
    return img2 - 1


def trans_mult_signed(A):
    B = normalize_image_signed(monochrome(A))
    result = normalize_image(np.dot(B, np.transpose(B)))
    return result

## Функции разрезания на ленты

### разрезание на ленты на примере одной матрицы

In [70]:
# А теперь возьмем m x n и n x k

num_rows_A = 7
num_columns_A = 9
strip_width = 2
number_strips = num_columns_A//strip_width +1
last_strip = num_columns_A%strip_width

A = np.random.randint(-10, 21, size=(num_rows_A, num_columns_A))

# Разделение матрицы A на number_strips матриц размером num_rows_A*strip_width и одну матрицу "остатка" num_rows*last_strip 
sub_matrices_A = np.array_split(A, number_strips, axis=1)




num_rows_B = 9
num_columns_B = 5
strip_width = 2
number_strips_B = num_rows_B//strip_width + 1
last_strip_B = num_rows_B%strip_width

B = np.random.randint(-3, 10, size=(num_rows_B, num_columns_B))


# Разделение матрицы A на number_strips матриц размером num_rows_A*strip_width и одну матрицу "остатка" num_rows*last_strip 
sub_matrices_B = np.array_split(B, number_strips_B, axis=0)

# размер получившихся "лент"




# Это у нас вторая вертикальная  "лента". Когда режем А
print(sub_matrices_A[1])
# Это у нас вторая горизонтальная  "лента". Когда режем В
print(sub_matrices_B[1])


print('форма вертикальных лент, нарезанных из матрицы А', sub_matrices_A[1].shape)
print('форма горизонтальных лент, нарезанных из матрицы В',sub_matrices_B[1].shape)
print()


C = [0] * len(sub_matrices_A)


for i in range(len(sub_matrices_A)):
    print('матрица С номер ',i+1,':')
    C[i] = sub_matrices_A[i].dot(sub_matrices_B[i])
    print(C[i])
    print()
    
# Если размеры матриц А и В m x n и n x m , то все леныт друг на друга легко умножаются 

[[ 0  3]
 [12  2]
 [ 9  7]
 [ 2 14]
 [-3 13]
 [ 2  5]
 [ 0  9]]
[[ 6  3  2  2  5]
 [ 4  5 -1  7 -2]]
форма вертикальных лент, нарезанных из матрицы А (7, 2)
форма горизонтальных лент, нарезанных из матрицы В (2, 5)

матрица С номер  1 :
[[  14   56   63    0   21]
 [  10   70   80  -45   45]
 [   2  -22  -26   45  -27]
 [  38   44   45  162  -51]
 [ -34  -94 -104  -63   -9]
 [  30   54   58   99  -21]
 [  12   18   19   45  -12]]

матрица С номер  2 :
[[ 12  15  -3  21  -6]
 [ 80  46  22  38  56]
 [ 82  62  11  67  31]
 [ 68  76 -10 102 -18]
 [ 34  56 -19  85 -41]
 [ 32  31  -1  39   0]
 [ 36  45  -9  63 -18]]

матрица С номер  3 :
[[  7  60 -46  -4 118]
 [ 18  33   3 111 -20]
 [ 10  42 -22  38  52]
 [ 15  39  -9  81  14]
 [ -2   9 -13 -25  36]
 [ -9 -24   6 -48 -10]
 [ -2   6 -10 -22  28]]

матрица С номер  4 :
[[ 43 -24 -18  55  67]
 [-32 -28 -50 -18  -4]
 [218  42 135 197 176]
 [-12  -2  -7 -11 -10]
 [ -2  28  38 -16 -30]
 [ 87   6  39  84  81]
 [100 -40 -20 120 140]]

матрица С ном

Пусть размеры матриц А и В ***m x n*** и ***n x m***

In [54]:
# разрежем одну матрицу на вертикальные ленты
# для разных размеров матрицы
# нужно ли менять название на  number_strips_A? Это по сути тот же number_strips, что и для B
num_rows_A = 7
num_columns_A = 9
strip_width = 2
number_strips_A = num_columns_A//strip_width+1
last_strip = num_columns_A%strip_width

A = np.random.randint(-10, 21, size=(num_rows_A, num_columns_A))



# Разделение матрицы A на number_strips матриц размером num_rows_A*strip_width и одну матрицу "остатка" num_rows*last_strip 
sub_matrices_A = np.array_split(A, number_strips_A, axis=1)

# размер получившихся "лент"
print(sub_matrices_A[0].shape)

# размер "ленты" остатка
print(sub_matrices_A[-1].shape)

# Это у нас первая "лента"

print(sub_matrices_A[0])
# это лента "остатка"
# sub_matrices_A[-1]

num_rows_B = 9
num_columns_B = 7
strip_width = 2
number_strips_B = num_rows_B//strip_width+1
last_strip_B = num_rows_B%strip_width

B = np.random.randint(-3, 10, size=(num_rows_B, num_columns_B))


# Разделение матрицы A на number_strips матриц размером num_rows_A*strip_width и одну матрицу "остатка" num_rows*last_strip 
sub_matrices_B = np.array_split(B, number_strips_B, axis=0)

# размер получившихся "лент"
print(sub_matrices_B[0].shape)

# размер "ленты" остатка
print(sub_matrices_B[-1].shape)
print(sub_matrices_B[0])

# Это у нас первая "лента"

# это лента "остатка"
# sub_matrices_B[-1]

C = [0] * len(sub_matrices_A)

for i in range(len(sub_matrices_A)):
    C[i] = sub_matrices_A[i].dot(sub_matrices_B[i])
    print(C[i])
    
# Если размеры матриц А и В m x n и n x m , то все леныт друг на друга легко умножаются 


(7, 2)
(7, 1)
[[17 -8]
 [-1 18]
 [16  2]
 [18 12]
 [16 -8]
 [-8 -2]
 [ 2 11]]
(2, 7)
(1, 7)
[[ 4 -1  6  6 -3  9  0]
 [-1 -2 -2  9  9  4  4]]
[[  76   -1  118   30 -123  121  -32]
 [ -22  -35  -42  156  165   63   72]
 [  62  -20   92  114  -30  152    8]
 [  60  -42   84  216   54  210   48]
 [  72    0  112   24 -120  112  -32]
 [ -30   12  -44  -66    6  -80   -8]
 [  -3  -24  -10  111   93   62   44]]
[[-22  26  -2  18  12  34  57]
 [ 72  54  22 -28  38 -34 -32]
 [-24 -15  -7  10 -11  13  14]
 [ 10 -20   0 -10 -10 -20 -35]
 [ 64  70  22 -20  46 -18  -4]
 [-14  19  -1  12   9  23  39]
 [ 78  51  23 -32  37 -41 -43]]
[[ 30  99   8  98  92  95  65]
 [114  33  48 126 156 141  27]
 [114  72  46 154 178 166  52]
 [-56  -8 -24 -56 -72 -64  -8]
 [ 99   3  43  91 121 106   7]
 [131  41  55 147 181 164  33]
 [-43 -34 -17 -63 -71 -67 -24]]
[[ 21  21  28 168  70  91 126]
 [ 72  48 -40 -24  32 -16   0]
 [-86 -55  61  87 -18  51  42]
 [ 10   5 -15 -45 -10 -25 -30]
 [ 68  46 -34  -6  36  -6  12]
 

Пусть  размеры матриц А и В ***m x n*** и ***n x k***

In [55]:
# разрежем одну матрицу на вертикальные ленты
# для разных размеров матрицы
# нужно ли менять название на  number_strips_A? Это по сути тот же number_strips, что и для B
num_rows_A = 7
num_columns_A = 9
strip_width = 2
number_strips_A = num_columns_A//strip_width+1


A = np.random.randint(-10, 21, size=(num_rows_A, num_columns_A))



# Разделение матрицы A на number_strips матриц размером num_rows_A*strip_width и одну матрицу "остатка" num_rows*last_strip 
sub_matrices_A = np.array_split(A, number_strips_A, axis=1)

# размер получившихся "лент"
print(sub_matrices_A[0].shape)

# размер "ленты" остатка
print(sub_matrices_A[-1].shape)

# Это у нас первая "лента"

print(sub_matrices_A[0])
# это лента "остатка"
# sub_matrices_A[-1]

num_rows_B = 9
num_columns_B = 5
strip_width = 2
number_strips_B = num_rows_B//strip_width+1


B = np.random.randint(-3, 10, size=(num_rows_B, num_columns_B))


# Разделение матрицы A на number_strips матриц размером num_rows_A*strip_width и одну матрицу "остатка" num_rows*last_strip 
sub_matrices_B = np.array_split(B, number_strips_B, axis=0)

# размер получившихся "лент"
print(sub_matrices_B[0].shape)

# размер "ленты" остатка
print(sub_matrices_B[-1].shape)
print(sub_matrices_B[0])

# Это у нас первая "лента"

# это лента "остатка"
# sub_matrices_B[-1]

C = [0] * len(sub_matrices_A)

for i in range(len(sub_matrices_A)):
    C[i] = sub_matrices_A[i].dot(sub_matrices_B[i])
    print(C[i])
    
# Если размеры матриц А и В m x n и n x m , то все леныт друг на друга легко умножаются 


(7, 2)
(7, 1)
[[ 3  0]
 [-1 -3]
 [14 11]
 [14 -6]
 [16  7]
 [18  4]
 [-5 10]]
(2, 5)
(1, 5)
[[ 3  8 -3  2  6]
 [ 0 -2  1 -3  9]]
[[  9  24  -9   6  18]
 [ -3  -2   0   7 -33]
 [ 42  90 -31  -5 183]
 [ 42 124 -48  46  30]
 [ 48 114 -41  11 159]
 [ 54 136 -50  24 144]
 [-15 -60  25 -40  60]]
[[-71  53  89 -66 -62]
 [106 101 -52 150 133]
 [ 45 219  57 114  96]
 [-12 167  86  34  25]
 [ 49  98  -1  84  73]
 [ -5  41  23   6   4]
 [ 46 152  26  96  82]]
[[-52 125  13  77  54]
 [ 60 -58  44 -48 -26]
 [-38 133  38  76  57]
 [-10 130  75  65  55]
 [ -8  85  47  43  36]
 [-30 124  43  69  53]
 [-36 -45 -81  -9 -18]]
[[ 12   9  -9  -3  12]
 [ -8  15  13  16 -15]
 [-20 -27  11  -3 -16]
 [-44 -48  28   1 -39]
 [128 147 -79   2 111]
 [100 108 -64  -3  89]
 [ 36   9 -33 -21  42]]
[[ -4  32  24   0   8]
 [ -1   8   6   0   2]
 [  3 -24 -18   0  -6]
 [-14 112  84   0  28]
 [ 10 -80 -60   0 -20]
 [-10  80  60   0  20]
 [ -8  64  48   0  16]]


In [11]:
t2_itogo = tm.time()
print('вся тетрадка заняла', round(t2_itogo - t1_itogo)//60,'минут', round(t2_itogo - t1_itogo)%60,'секунд')

вся тетрадка заняла 0 минут 0 секунд
